## 数据处理

In [24]:
from MLFeatureSelection import sequence_selection as ss
from sklearn.metrics import log_loss
import pandas as pd
import lightgbm as lgbm
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [25]:
data = pd.read_table('car.data',sep=',',names=['buying','maint','doors','persons','lug_boot','safety' ,'value'])

In [26]:
doors = list(np.unique(data.doors))
data.doors.replace(doors, list(np.arange(len(doors))), inplace=True)

In [27]:
buying = list(np.unique(data.buying))
data.buying.replace(buying, list(np.arange(len(buying))), inplace=True)

In [28]:
persons = list(np.unique(data.persons))
data.persons.replace(persons, list(np.arange(len(persons))), inplace=True)

In [29]:
lug_boot = list(np.unique(data.lug_boot))
data.lug_boot.replace(lug_boot, list(np.arange(len(lug_boot))), inplace=True)

In [30]:
maint = list(np.unique(data.maint))
data.maint.replace(maint, list(np.arange(len(maint))), inplace=True)

In [31]:
safety = list(np.unique(data.safety))
data.safety.replace(safety, list(np.arange(len(safety))), inplace=True)

In [32]:
value = list(np.unique(data.value))
data.value.replace(value, list(np.arange(len(value))), inplace=True)

In [33]:
data

,buying,maint,doors,persons,lug_boot,safety,value
0,3,3,0,0,2,1,2
1,3,3,0,0,2,2,2
2,3,3,0,0,2,0,2
3,3,3,0,0,1,1,2
4,3,3,0,0,1,2,2
...,...,...,...,...,...,...,...
1723,1,1,3,2,1,2,1
1724,1,1,3,2,1,0,3
1725,1,1,3,2,0,1,2
1726,1,1,3,2,0,2,1


## 函数定义

In [34]:
def lossfunction(y_test, y_pred):
    """set up the evaluation score"""
    return np.mean(np.abs(y_pred - y_test)/y_test)

def validation(X,y,features, clf,lossfunction):
    totaltest = []
    kf = KFold(6)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index,:][features], X.iloc[test_index,:][features]
        y_train, y_test = y[train_index], y[test_index]
        #clf.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], eval_metric='logloss', verbose=False,early_stopping_rounds=50)
        clf.fit(X_train, y_train)
        totaltest.append(lossfunction(y_test, clf.predict(X_test)))
    return np.mean(totaltest), clf

def add(x,y):
    return x + y

def substract(x,y):
    return x - y

def times(x,y):
    return x * y

def divide(x,y):
    return (x + 0.001)/(y + 0.001)

CrossMethod = {'+':add,
               '-':substract,
               '*':times,
               '/':divide,}

## 模块测试

In [35]:
sf = ss.Select(Sequence = True, Random = False, Cross = True)
sf.ImportDF(data,label = 'value')
sf.ImportLossFunction(lossfunction,direction = 'ascend')
sf.ImportCrossMethod(CrossMethod)
sf.InitialFeatures(['buying','maint','doors','persons','lug_boot','safety','value'])
sf.GenerateCol()
sf.SetSample(0.5, samplemode = 0, samplestate = 0)
sf.clf = LogisticRegression()
sf.SetLogFile('record2.log')
#sf.SetFeaturesLimit(5)
sf.SetTimeLimit(0.5)
sf.run(validation)

Features Quantity Limit: inf
Time Limit: 0.5 min(s)
100000000
test performance of initial features combination
Mean loss: 0.019401904727991682


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


--------------------start greedy--------------------
buying
maint
doors
persons
lug_boot
safety
value
******************** 8 round ********************
buying
reverse 0/6
Mean loss: 0.019401904727991682
maint
reverse 1/6


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mean loss: 0.014883993960080916
doors
reverse 2/6
Mean loss: 0.019401904727991682
persons
reverse 3/6
Mean loss: 0.01639865797474493
lug_boot
reverse 4/6
Mean loss: 0.012455817618861097
safety
reverse 5/6
Mean loss: 0.02290118322727018
******************** 7 round ********************
safety
0/1
Mean loss: 0.019401904727991682
buying
reverse 0/5


...

Mean loss: 0.08265993089362654
(buying*buying)
reverse 9/11


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_ms

Mean loss: inf
(buying/buying)
reverse 10/11
Mean loss: inf
--------------------complete greedy--------------------
random select starts with:
 ['buying', 'maint', 'persons', 'lug_boot', 'value', '(buying/lug_boot)', '(buying+persons)', '(buying/maint)', '(buying+buying)', '(buying*buying)', '(buying/buying)', '(maint+value)']
 score: inf
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
best score:inf
best features combination: ['buying', 'maint', 'persons', 'lug_boot', 'value', '(buying/lug_boot)', '(buying+persons)', '(buying/maint)', '(buying+buying)', '(buying*buying)', '(buying/buying)', '(maint+value)']


E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
E:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_ms

['buying',
 'maint',
 'persons',
 'lug_boot',
 'value',
 '(buying/lug_boot)',
 '(buying+persons)',
 '(buying/maint)',
 '(buying+buying)',
 '(buying*buying)',
 '(buying/buying)',
 '(maint+value)']

In [23]:
from MLFeatureSelection import importance_selection
import xgboost as xgb


sf = importance_selection.Select() #initialized selector
sf.ImportDF(data,label = 'value') #import dataset
sf.ImportLossFunction(lossfunction, direction = 'descend') #import loosfunction and improve direction
sf.InitialFeatures(['buying','maint','doors','persons','lug_boot','safety','value'])
sf.SelectRemoveMode(batch = 1)
sf.clf = lgbm.LGBMRegressor(num_leaves=35, max_depth=-1)
sf.SetLogFile('record_imp.log') #set the log file name
sf.run(validation) #start running

Features Quantity Limit: inf
Time Limit: inf min(s)
test performance of initial features combination
remove features: baseline
Mean loss: inf
Remove Batch: 1
remove features: ['safety']
Mean loss: inf
remove features: ['lug_boot']
Mean loss: inf
remove features: ['persons']
Mean loss: inf
remove features: ['maint']
Mean loss: inf
remove features: ['doors']
Mean loss: inf
remove features: ['buying']
Mean loss: inf
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
best score:inf
best features combination: ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'value']


['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'value']

In [18]:
from MLFeatureSelection import coherence_selection
import xgboost as xgb

sf = coherence_selection.Select() #initialized selector
sf.ImportDF(data,label = 'value') #import dataset
sf.ImportLossFunction(lossfunction, direction = 'descend') #import loosfunction and improve direction 
sf.InitialFeatures(['buying','maint','doors','persons','lug_boot','safety','value'])
sf.SelectRemoveMode(batch = 1)
sf.clf = xgb.XGBClassifier() 
sf.SetLogFile('record_coh.log') #set the log file name
sf.run(validation) #start running

Features Quantity Limit: inf
Time Limit: inf min(s)
test performance of initial features combination
remove features: baseline
Mean loss: 0.0
Remove Batch: 1
totally 0 features above 0.8
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
best score:inf
best features combination: ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'value']


['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'value']